In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# reading train and test data 
application_train = pd.read_csv('../input/application_train.csv')
application_test = pd.read_csv('../input/application_test.csv')

In [ ]:
#displaying shape
print('train shape:', application_train.shape)
print('test shape:', application_test.shape)

In [ ]:
application_train.describe()

## Handling Bureau Data

In [ ]:
bureau = pd.read_csv('../input/bureau.csv')
bureau_balance = pd.read_csv('../input/bureau_balance.csv')

In [ ]:
print('bureau shape:', bureau.shape)
print('bureau balance shape:', bureau_balance.shape)
bureau.describe()

In [ ]:
print('total entry in bureau:', bureau.shape[0])
print('unique values in Application train:', len(application_train['SK_ID_CURR'].unique()), 'unique value in bureau:', len(bureau['SK_ID_CURR'].unique()))
print('avg number of loan taken by person', bureau.shape[0] / len(bureau['SK_ID_CURR'].unique()) )

In [ ]:
applicationColumn = application_train.columns.values
bureauColumn = bureau.columns.values
print(set(applicationColumn) & set(bureauColumn))

In [ ]:
bureau_numeric = bureau.select_dtypes(include=["number"])
bureau_numeric = bureau_numeric.drop('SK_ID_BUREAU', axis = 1)
bureau_numeric.head()

In [ ]:
bureau_categorical = pd.get_dummies(bureau.select_dtypes(exclude = ["number"]))
bureau_categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
bureau_categorical.head()

In [ ]:
def splitData(dataframe):
    df_numeric_data =dataframe.select_dtypes(include=["number"])
    df_categorical_data = dataframe.select_dtypes(exclude = ["number"])
    return df_numeric_data, df_categorical_data

In [ ]:
def handleNumericData(df, withColumn):
    df.fillna(0,inplace = True)
    ag = df.groupby(withColumn).agg(['count', 'mean'])
    ag.columns = ag.columns.map('_'.join).str.strip('_')
    ag = ag.reset_index()
    return ag


def handleCategoricalData(df, withColumn):
    ag = df.groupby(withColumn).agg(['sum', 'mean'])
    ag.columns = ag.columns.map('_'.join).str.strip('_')
    ag = ag.reset_index()
    return ag

bureau_categorical = handleCategoricalData(bureau_categorical, 'SK_ID_CURR')
bureau_numeric = handleNumericData(bureau_numeric, 'SK_ID_CURR')

In [ ]:
bureau_updated = bureau_numeric.merge(bureau_categorical, on = 'SK_ID_CURR')

In [ ]:
bureau_updated.head()

# Extracting from bureau balance

In [ ]:
# 

bureau_balance_numeric = bureau_balance.select_dtypes(include=["number"])
bureau_balance_numeric = handleNumericData(bureau_balance_numeric, 'SK_ID_BUREAU')

bureau_balance_categorical = pd.get_dummies(bureau_balance.select_dtypes(exclude=["number"]))
bureau_balance_categorical['SK_ID_BUREAU'] = bureau_balance['SK_ID_BUREAU']
bureau_balance_categorical = handleCategoricalData(bureau_balance_categorical, 'SK_ID_BUREAU')


In [ ]:
bureau_balance = bureau_balance_numeric.merge(bureau_balance_categorical, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')
bureau_balance = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_balance, on = 'SK_ID_BUREAU', how = 'left')
bureau_balance_by_client = handleNumericData(bureau_balance.drop(columns = ['SK_ID_BUREAU']),'SK_ID_CURR')

In [ ]:
bureau_balance_by_client.head()

# Combining Bureau and Bureau Balance data

In [ ]:
bureau_combined = bureau_updated.merge(bureau_balance_by_client, on = 'SK_ID_CURR')
print('bureau_combined shape:', bureau_combined.shape)
bureau_combined.head()

# Combining Application and Combined Data

In [ ]:
application_train_updated = application_train.merge(bureau_combined, on = 'SK_ID_CURR')
application_test_updated = application_test.merge(bureau_combined, on = 'SK_ID_CURR')

print('application_train_updated shape:', application_train_updated.shape)
print('application_test_updated shape:', application_test_updated.shape)

application_train_updated.head()


In [ ]:
application_train_updated.head(20)

In [ ]:
#starting with reading bureau and bureau balance

bureau = pd.read_csv('../input/bureau.csv')
bureau_balance = pd.read_csv('../input/bureau_balance.csv')

bureau_combined = bureau.merge(bureau_balance,on = 'SK_ID_BUREAU', how = 'outer')

In [ ]:
import gc
del [[bureau,bureau_balance]]
gc.collect()

In [ ]:
bureau_combined.head()

In [ ]:
corr_matrix = bureau_combined.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.6)]
#print(to_drop)
bureau_combined.drop(to_drop, axis=1, inplace=True)

In [ ]:
import gc
del [[upper,corr_matrix]]
gc.collect()

In [ ]:
bureau_combined.head()

In [ ]:
import scipy.stats
def splitData(dataframe, idColumn):
    df_numeric_data =dataframe.select_dtypes(include=["number"])
    df_categorical_data = pd.get_dummies(dataframe.select_dtypes(exclude = ["number"]))
    df_categorical_data[idColumn] = dataframe[idColumn]
    return df_numeric_data, df_categorical_data

def handleNumericData(df, withColumn):
    df.fillna(0,inplace = True)
    ag = df.groupby(withColumn).agg(['mean', 'min', 'max'])
    ag.columns = ag.columns.map('_'.join).str.strip('_')
    ag = ag.reset_index()
    return ag


def handleCategoricalData(df, withColumn):
    
    '''maxCategorical = categorical.groupby(withColumn).agg(lambda x: scipy.stats.mode(x)[0][0])
    maxCategorical = maxCategorical.reset_index()
    #maxCategorical.drop(withColumn,inplace = True)'''
    
    ag = df.groupby(withColumn).agg(['sum'])
    ag.columns = ag.columns.map('_'.join).str.strip('_')
    ag = ag.reset_index()
    #ag = pd.concat([ag, maxCategorical], axis = 1)
    #ag['Max'] = ag[list(ag.columns[1:])].max(axis=1)

    return ag

def treatDataFrame(dataframe, idColumn):
    numerical, categorical = splitData(dataframe, idColumn)
    #del [[df_numeric_data,df_categorical_data]]
    gc.collect()    
    
    numerical = handleNumericData(numerical, idColumn)
    #del [[ag]]
    gc.collect()    
    
    categorical = handleCategoricalData(categorical, idColumn)
    #del [[ag]]
    gc.collect()
    
    categorical.drop(idColumn,axis = 1, inplace= True)
    combined_data = pd.concat([numerical, categorical], axis = 1)
    del [[numerical,categorical]]
    gc.collect()
    return combined_data
    

In [ ]:
# handleCategoricalData(categorical.iloc[:10,:],'SK_ID_BUREAU')
# #categorical.head()
#bureau_balance.head()
bureau_combined.drop('SK_ID_BUREAU', axis = 1, inplace = True)
bureau_combined = treatDataFrame(bureau_combined, 'SK_ID_CURR')
bureau_combined.head()

In [ ]:
bureau_combined.to_csv('../bureau_combined.csv')

In [ ]:
sk_id = bureau['SK_ID_CURR']
bureau.drop('SK_ID_CURR',axis = 1, inplace = True)

bureau = treatDataFrame(bureau, 'SK_ID_BUREAU')
bureau['SK_ID_CURR'] = sk_id
bureau.head()

In [ ]:
import scipy.stats

maxCategorical.head()

In [ ]:
categorical_dummies = pd.get_dummies(categorical)
categorical_dummies['SK_ID_BUREAU'] = bureau_balance['SK_ID_BUREAU']
categorical_dummies.head()

In [ ]:
#ag = categorical_dummies.groupby('SK_ID_BUREAU').agg(['sum'])
#agMode = categorical.groupby('SK_ID_BUREAU').agg(lambda x: x.value_counts().index[0])
handleCategoricalData(categorical_dummies, 'SK_ID_BUREAU').head()


# ag.columns = ag.columns.map('_'.join).str.strip('_')
# ag = ag.reset_index()
# ag['Max'] = ag[list(ag.columns[1:])].max(axis=1)
# ag.head()

In [ ]:
ag.head()

In [ ]:
handleCategoricalData(categorical, 'SK_ID_BUREAU')

Working on data

In [103]:
train = pd.read_csv('../input/train-all/train_all.csv')
bureau_combined = pd.read_csv('../input/bureau/bureau_combined.csv')


In [104]:
print(train.shape)
print(bureau_combined.shape)

(263491, 593)
(305812, 59)


In [105]:
train.head()

,Unnamed: 0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,...,AMT_CREDIT_SUM_LIMIT_mean,AMT_CREDIT_SUM_LIMIT_min,AMT_CREDIT_SUM_LIMIT_max,AMT_CREDIT_SUM_OVERDUE_mean,AMT_CREDIT_SUM_OVERDUE_min,AMT_CREDIT_SUM_OVERDUE_max,AMT_ANNUITY_mean_y.1,AMT_ANNUITY_min_y.1,AMT_ANNUITY_max_y.1,CREDIT_ACTIVE_Active_sum,CREDIT_ACTIVE_Bad debt_sum,CREDIT_ACTIVE_Closed_sum,CREDIT_ACTIVE_Sold_sum,CREDIT_CURRENCY_currency 1_sum,CREDIT_CURRENCY_currency 2_sum,CREDIT_CURRENCY_currency 3_sum,CREDIT_CURRENCY_currency 4_sum,CREDIT_TYPE_Another type of loan_sum,CREDIT_TYPE_Car loan_sum,CREDIT_TYPE_Cash loan (non-earmarked)_sum,CREDIT_TYPE_Consumer credit_sum,CREDIT_TYPE_Credit card_sum,CREDIT_TYPE_Interbank credit_sum,CREDIT_TYPE_Loan for business development_sum,CREDIT_TYPE_Loan for purchase of shares (margin lending)_sum,CREDIT_TYPE_Loan for the purchase of equipment_sum,CREDIT_TYPE_Loan for working capital replenishment_sum,CREDIT_TYPE_Microloan_sum,CREDIT_TYPE_Mobile operator loan_sum,CREDIT_TYPE_Mortgage_sum,CREDIT_TYPE_Real estate loan_sum,CREDIT_TYPE_Unknown type of loan_sum,STATUS_0_sum,STATUS_1_sum,STATUS_2_sum,STATUS_3_sum,STATUS_4_sum,STATUS_5_sum,STATUS_C_sum,STATUS_X_sum
0,0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,1.0,2,2,10,0,0,0,0,0,0,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,...,1163.220545,0.0,31988.565,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,2.0,1,1,11,0,0,0,0,0,0,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,...,202500.000000,0.0,810000.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,1.0,2,2,9,0,0,0,0,0,0,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,13.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0
3,3,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,1.0,2,2,11,0,0,0,0,1,1,NaN,0.322738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100008,0,0,99000.0,490495.5,27517.5,454500.0,0.035792,-16941,-1588,-4970.0,-477,NaN,1,1,1,1,1,0,2.0,2,2,16,0,0,0,0,0,0,NaN,0.354225,0.621226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,14.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
target = train['TARGET']
train.drop(['SK_ID_CURR', 'TARGET', 'Unnamed: 0'],axis = 1, inplace = True)

In [107]:
number = train.isnull().sum().sort_values(ascending = False)
percent = (train.isnull().sum() / train.isnull().count() * 100).sort_values(ascending = False)

missing_application_test = pd.concat([number , percent] , axis = 1 , keys = ['Total' , 'Percent'])
print(missing_application_test.shape)
print(missing_application_test.head(20))

(590, 2)
                           Total    Percent
COMMONAREA_AVG            183219  69.535202
COMMONAREA_MODE           183219  69.535202
COMMONAREA_MEDI           183219  69.535202
NONLIVINGAPARTMENTS_MODE  182038  69.086990
NONLIVINGAPARTMENTS_MEDI  182038  69.086990
NONLIVINGAPARTMENTS_AVG   182038  69.086990
LIVINGAPARTMENTS_MODE     179191  68.006497
LIVINGAPARTMENTS_MEDI     179191  68.006497
LIVINGAPARTMENTS_AVG      179191  68.006497
FLOORSMIN_AVG             177890  67.512742
FLOORSMIN_MODE            177890  67.512742
FLOORSMIN_MEDI            177890  67.512742
YEARS_BUILD_AVG           174239  66.127116
YEARS_BUILD_MODE          174239  66.127116
YEARS_BUILD_MEDI          174239  66.127116
OWN_CAR_AGE               171716  65.169588
LANDAREA_MEDI             155200  58.901443
LANDAREA_AVG              155200  58.901443
LANDAREA_MODE             155200  58.901443
BASEMENTAREA_MEDI         153004  58.068018


In [108]:
train_full = train.loc[:, train.isnull().mean() < .5]

In [109]:
print(train.shape)
print(train_full.shape)

(263491, 590)
(263491, 558)


In [110]:
train_full.fillna(train_full.mean(),inplace = True)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [ ]:
train_full.shape

In [ ]:
y_predict = rdf.predict(X_train)
from sklearn.metrics import precision_recall_fscore_support
print('precision_recall_fscore_support score:', precision_recall_fscore_support(y_predict, y_train))

In [111]:
import gc
test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
numerical, categorical = splitData(test, 'SK_ID_CURR')
print(test.shape)
test  = pd.merge(numerical,categorical, on = 'SK_ID_CURR', how = 'inner')
test.head()

(48744, 121)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,...,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,2.0,2,2,18,0,0,0,0,0,0,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,2.0,2,2,9,0,0,0,0,0,0,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,2.0,2,2,14,0,0,0,0,0,0,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866,-2000.0,-4208,NaN,1,1,0,1,1,0,4.0,2,2,11,0,0,0,0,0,0,0.525734,0.509677,0.612704,0.3052,0.1974,0.9970,0.9592,0.1165,0.32,0.2759,0.375,0.0417,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191,-4000.0,-4262,16.0,1,1,1,1,0,0,3.0,2,2,5,0,0,0,0,1,1,0.202145,0.425687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [112]:
test  = pd.merge(test,bureau_combined, on = 'SK_ID_CURR', how = 'left')
print(test.shape)

(48744, 300)


In [113]:
common_columns = list(set(test.columns) & set(train_full.columns)) 


In [114]:
train_common = train_full[common_columns]

In [115]:
print(train_common.shape)
print(target.shape)

(263491, 264)
(263491,)


In [123]:
train_common.head()

,AMT_REQ_CREDIT_BUREAU_DAY,AMT_CREDIT_MAX_OVERDUE_mean,NAME_FAMILY_STATUS_Separated,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Other,STATUS_3_sum,FLAG_DOCUMENT_9,OCCUPATION_TYPE_Private service staff,FLAG_PHONE,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Self-employed,"NAME_TYPE_SUITE_Spouse, partner",NAME_INCOME_TYPE_State servant,NAME_FAMILY_STATUS_Civil marriage,ORGANIZATION_TYPE_Industry: type 13,CREDIT_TYPE_Interbank credit_sum,YEARS_BEGINEXPLUATATION_MEDI,CREDIT_TYPE_Microloan_sum,CREDIT_TYPE_Consumer credit_sum,EMERGENCYSTATE_MODE_Yes,ORGANIZATION_TYPE_Industry: type 4,NAME_HOUSING_TYPE_Municipal apartment,OCCUPATION_TYPE_Low-skill Laborers,FONDKAPREMONT_MODE_reg oper account,FLAG_OWN_REALTY_Y,FLAG_DOCUMENT_21,YEARS_BEGINEXPLUATATION_MODE,STATUS_5_sum,DAYS_ID_PUBLISH,ORGANIZATION_TYPE_Services,FLAG_DOCUMENT_4,ORGANIZATION_TYPE_Industry: type 1,CREDIT_CURRENCY_currency 4_sum,CREDIT_TYPE_Car loan_sum,CODE_GENDER_F,CREDIT_DAY_OVERDUE_max,AMT_REQ_CREDIT_BUREAU_QRT,CNT_CREDIT_PROLONG_max,STATUS_2_sum,ORGANIZATION_TYPE_Legal Services,...,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Industry: type 3,DAYS_CREDIT_ENDDATE_max,FLAG_OWN_CAR_Y,CREDIT_ACTIVE_Active_sum,NAME_EDUCATION_TYPE_Lower secondary,WALLSMATERIAL_MODE_Mixed,AMT_CREDIT,ORGANIZATION_TYPE_Postal,DAYS_REGISTRATION,FLAG_EMP_PHONE,EXT_SOURCE_3,OCCUPATION_TYPE_Security staff,CNT_FAM_MEMBERS,DEF_30_CNT_SOCIAL_CIRCLE,ORGANIZATION_TYPE_Construction,REGION_POPULATION_RELATIVE,FLAG_DOCUMENT_6,HOUSETYPE_MODE_terraced house,CREDIT_CURRENCY_currency 3_sum,STATUS_4_sum,FLOORSMAX_MEDI,FLOORSMAX_AVG,NAME_HOUSING_TYPE_Office apartment,AMT_CREDIT_SUM_LIMIT_max,WALLSMATERIAL_MODE_Monolithic,NAME_HOUSING_TYPE_With parents,STATUS_0_sum,OCCUPATION_TYPE_Cleaning staff,ORGANIZATION_TYPE_Realtor,STATUS_1_sum,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Security,DAYS_CREDIT_ENDDATE_min,CREDIT_ACTIVE_Bad debt_sum,NAME_INCOME_TYPE_Commercial associate,OCCUPATION_TYPE_Realty agents,FLAG_EMAIL,NAME_INCOME_TYPE_Working,ORGANIZATION_TYPE_Medicine
0,0.0,667.932545,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0.0,0.972200,0.0,2.0,0,0,0,0,1,1,0,0.972200,0.0,-2120,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0,0,780.0,0,1.0,0,0,406597.5,0,-3648.0,1,0.139376,0,1.0,2.0,0,0.018801,0,0,0.0,0.0,0.083300,0.083300,0,31988.565,0,0,0.0,0,0,0.0,0,0,-1072.0,0.0,0,0,0,1,0
1,0.0,0.000000,0,0,0,0.0,0,0,1,0,0,0,1,0,0,0.0,0.985100,0.0,2.0,0,0,0,0,1,0,0,0.985100,0.0,-291,0,0,0,0.0,0.0,1,0.0,0.0,0.0,0.0,0,...,0,0,1216.0,0,0.0,0,0,1293502.5,0,-1186.0,1,0.509755,0,2.0,0.0,0,0.003541,0,0,0.0,0.0,0.291700,0.291700,0,810000.000,0,0,0.0,0,0,0.0,0,0,-2434.0,0.0,0,0,0,0,0
2,0.0,0.000000,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0.0,0.977887,0.0,8.0,0,0,0,0,0,1,0,0.977191,0.0,-2531,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0,0,-382.0,1,8.0,0,0,135000.0,0,-4260.0,1,0.729567,0,1.0,0.0,0,0.010032,0,0,0.0,0.0,0.225797,0.226171,0,0.000,0,0,14.0,0,0,0.0,0,0,-595.0,0.0,0,0,0,1,0
3,0.0,0.000000,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.977887,0.0,3.0,0,0,0,0,0,1,0,0.977191,0.0,-3458,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0,0,-783.0,0,1.0,0,0,513000.0,0,-4311.0,1,0.509755,0,1.0,0.0,0,0.028663,0,0,0.0,0.0,0.225797,0.226171,0,0.000,0,0,0.0,0,0,0.0,0,0,-783.0,0.0,0,0,0,1,0
4,0.0,0.000000,0,0,1,0.0,0,0,1,0,0,1,1,0,0,0.0,0.977887,0.0,16.0,0,0,0,0,0,1,0,0.977191,0.0,-477,0,0,0,0.0,0.0,0,0.0,1.0,0.0,0.0,0,...,0,0,471.0,0,4.0,0,0,490495.5,0,-4970.0,1,0.621226,0,2.0,0.0,0,0.035792,0,0,0.0,0.0,0.225797,0.226171,0,0.000,0,0,0.0,0,0,0.0,0,0,-853.0,0.0,0,0,0,0,0


In [116]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(train_common, target.ravel())

Using TensorFlow backend.


In [117]:
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

After OverSampling, the shape of train_X: (486246, 264)
After OverSampling, the shape of train_y: (486246,) 

After OverSampling, counts of label '1': 243123
After OverSampling, counts of label '0': 243123


In [125]:
from sklearn.ensemble import RandomForestClassifier
rdf = RandomForestClassifier(n_estimators = 100 , max_depth = 5, min_samples_leaf = 4, max_features = 0.5)
rdf.fit(X_train_res , y_train_res)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [129]:
print(len(test['SK_ID_CURR']))
test.fillna(test.mean(),inplace = True)


48744


In [124]:
test[common_columns].head()

,AMT_REQ_CREDIT_BUREAU_DAY,AMT_CREDIT_MAX_OVERDUE_mean,NAME_FAMILY_STATUS_Separated,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Other,STATUS_3_sum,FLAG_DOCUMENT_9,OCCUPATION_TYPE_Private service staff,FLAG_PHONE,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Self-employed,"NAME_TYPE_SUITE_Spouse, partner",NAME_INCOME_TYPE_State servant,NAME_FAMILY_STATUS_Civil marriage,ORGANIZATION_TYPE_Industry: type 13,CREDIT_TYPE_Interbank credit_sum,YEARS_BEGINEXPLUATATION_MEDI,CREDIT_TYPE_Microloan_sum,CREDIT_TYPE_Consumer credit_sum,EMERGENCYSTATE_MODE_Yes,ORGANIZATION_TYPE_Industry: type 4,NAME_HOUSING_TYPE_Municipal apartment,OCCUPATION_TYPE_Low-skill Laborers,FONDKAPREMONT_MODE_reg oper account,FLAG_OWN_REALTY_Y,FLAG_DOCUMENT_21,YEARS_BEGINEXPLUATATION_MODE,STATUS_5_sum,DAYS_ID_PUBLISH,ORGANIZATION_TYPE_Services,FLAG_DOCUMENT_4,ORGANIZATION_TYPE_Industry: type 1,CREDIT_CURRENCY_currency 4_sum,CREDIT_TYPE_Car loan_sum,CODE_GENDER_F,CREDIT_DAY_OVERDUE_max,AMT_REQ_CREDIT_BUREAU_QRT,CNT_CREDIT_PROLONG_max,STATUS_2_sum,ORGANIZATION_TYPE_Legal Services,...,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Industry: type 3,DAYS_CREDIT_ENDDATE_max,FLAG_OWN_CAR_Y,CREDIT_ACTIVE_Active_sum,NAME_EDUCATION_TYPE_Lower secondary,WALLSMATERIAL_MODE_Mixed,AMT_CREDIT,ORGANIZATION_TYPE_Postal,DAYS_REGISTRATION,FLAG_EMP_PHONE,EXT_SOURCE_3,OCCUPATION_TYPE_Security staff,CNT_FAM_MEMBERS,DEF_30_CNT_SOCIAL_CIRCLE,ORGANIZATION_TYPE_Construction,REGION_POPULATION_RELATIVE,FLAG_DOCUMENT_6,HOUSETYPE_MODE_terraced house,CREDIT_CURRENCY_currency 3_sum,STATUS_4_sum,FLOORSMAX_MEDI,FLOORSMAX_AVG,NAME_HOUSING_TYPE_Office apartment,AMT_CREDIT_SUM_LIMIT_max,WALLSMATERIAL_MODE_Monolithic,NAME_HOUSING_TYPE_With parents,STATUS_0_sum,OCCUPATION_TYPE_Cleaning staff,ORGANIZATION_TYPE_Realtor,STATUS_1_sum,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Security,DAYS_CREDIT_ENDDATE_min,CREDIT_ACTIVE_Bad debt_sum,NAME_INCOME_TYPE_Commercial associate,OCCUPATION_TYPE_Realty agents,FLAG_EMAIL,NAME_INCOME_TYPE_Working,ORGANIZATION_TYPE_Medicine
0,0.0,0.000000,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.9732,0.0,52.0,0,0,0,0,0,1,0,0.9732,0.0,-812,0,0,0,0.0,0.0,1,0.0,0.0,0.0,0.0,0,...,0,0,1778.0,0,20.0,0,0,568800.0,0,-5170.0,1,0.159520,0,2.0,0.0,0,0.018850,0,0,0.0,0.0,0.125,0.125,0,0.00,0,0,45.0,0,0,27.0,0,0,-1329.0,0.0,0,0,1,1,0
1,0.0,0.000000,0,0,0,0.0,0,0,0,0,1,0,0,0,0,0.0,NaN,0.0,1.0,0,0,0,1,0,1,0,NaN,0.0,-1623,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0,0,1324.0,0,0.0,0,0,222768.0,0,-9118.0,1,0.432962,0,2.0,0.0,0,0.035792,0,0,0.0,0.0,NaN,NaN,0,0.00,0,0,0.0,0,0,0.0,0,0,-128.0,0.0,0,0,0,1,0
2,0.0,5539.695652,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,NaN,0.0,8.0,0,0,0,0,0,1,0,NaN,0.0,-3503,0,0,0,0.0,0.0,0,0.0,1.0,0.0,0.0,0,...,0,0,-567.0,1,2.0,0,0,663264.0,0,-2175.0,1,0.610991,0,2.0,0.0,0,0.019101,0,0,0.0,0.0,NaN,NaN,0,0.00,0,0,0.0,0,0,0.0,0,0,-1707.0,0.0,0,0,0,1,0
3,0.0,0.000000,0,0,0,0.0,0,0,1,0,0,0,0,0,0,0.0,0.9970,0.0,2.0,0,0,0,0,1,1,0,0.9970,0.0,-4208,0,0,0,0.0,0.0,1,0.0,0.0,0.0,0.0,0,...,0,0,30885.0,0,1.0,0,0,1575000.0,0,-2000.0,1,0.612704,0,4.0,0.0,0,0.026392,0,0,0.0,0.0,0.375,0.375,0,101390.76,0,0,0.0,0,0,0.0,0,0,-1862.0,0.0,0,0,0,1,0
4,NaN,NaN,0,0,0,NaN,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,NaN,-4262,0,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,0,0,NaN,1,NaN,0,0,625500.0,0,-4000.0,1,NaN,0,3.0,0.0,0,0.010032,0,0,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0,0,1,0


In [130]:
#X_train_res = sm.fit_sample(test[common_columns])
y_pred_sample_score = rdf.predict(test[common_columns])


In [132]:
len(y_pred_sample_score)

48744

In [135]:
df = pd.DataFrame()
df['SK_ID_CURR'] = test['SK_ID_CURR']
df['TARGET'] = y_pred_sample_score
df.to_csv('sample_submission.csv', index=False)

In [136]:
df.head()

,SK_ID_CURR,TARGET
0,100001,0
1,100005,1
2,100013,0
3,100028,0
4,100038,1


In [137]:
sub = pd.read_csv('../input/home-credit-default-risk/sample_submission.csv')
sub['TARGET'] = y_pred_sample_score
sub.to_csv('submission2.csv',index = False)

In [138]:
sub.head()

,SK_ID_CURR,TARGET
0,100001,0
1,100005,1
2,100013,0
3,100028,0
4,100038,1
